Данные, которые будем использовать в качестве примера

In [13]:
import pandas as pd

# Прочитаем данные из файла и выберем в качестве факторов все числовые столбцы
data = pd.read_csv('data\insurance.csv')
num_columns = [s for s in data.columns.drop('charges') if data[s].dtypes != 'object']
X = data[num_columns]
# В качестве целевого выберем столбец charges
y = data['charges']

# Регрессия

**Регрессия — это класс задач обучения с учителем, когда по определённому набору признаков объекта необходимо предсказать числовую целевую переменную.**
$$  h(x) = \sum_{i=0}^{d}\theta_ix_i = \vec{\theta}^T\vec{x}  $$
$h$ — функция (модель), прогнозирующая значения;\
$\theta_i$ — параметры;\
$x_i$ — факторы (по умолчанию $x_0 = 1$).

## Метод наименьших квадратов

Функция потерь:
$$ J(\theta) = \frac{1}{2}\sum_{i=1}^{n}(h(\vec{x}^{(i)}) - y^{(i)})^2 $$
$\vec{x}^{(i)}$ — $i$-ый вектор факторов в выборке;\
$y^{(i)}$ — $i$-ое значение целевого параметра в выборке.

### Аналитическое решение

$$ \theta = (X^TX)^{-1}X^T\vec{y} $$
#### Преимущества
* Если выполняются условия теоремы Гаусса-Маркова, всегда находит оптимальные оценки параметров.
#### Недостатки
* Увеличение сложности расчетов с ростом количества признаков ($n^3$).
* Невозможность инкрементального обучения (обучения в режиме реального времени).

In [17]:
from sklearn import linear_model

#Создаём объект класса LinearRegression
lr_lstat = linear_model.LinearRegression()
#Обучаем модель — ищем параметры по МНК
lr_lstat.fit(X, y)
 
print('w0: {}'.format(lr_lstat.intercept_)) #свободный член w0
print('w1: {}'.format(lr_lstat.coef_)) #остальные параметры модели w1, w2, ..., wm

#Предсказываем медианную цену для всех участков из набора данных
y_predict = lr_lstat.predict(X)
y_predict[:5]

w0: -6916.243347787033
w1: [239.99447429 332.0833645  542.86465225]


array([ 6908.77753344,  9160.97706103, 12390.94691779,  8543.52709495,
       10354.14739647])

## Метрики

**Метрика — это численное выражение качества моделирования.**

### Средняя абсолютная ошибка — MAE (Mean Absolute Error)
$$ MAE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y_i}) $$
#### Преимущества
* Удобно интерпретировать.
* Измеряется в тех же единицах, что и целевой признак.
* Несильно искажается при наличии выбросов.
#### Недостатки
* Не поможет, если необходимо сравнить модели, предсказывающие одно и то же по разным признакам.

In [19]:
from sklearn import metrics

#Рассчитываем MAE
print('MAE score: {:.3f}'.format(metrics.mean_absolute_error(y, y_predict)))

MAE score: 9015.442


### Средняя абсолютная ошибка в процентах — MAPE (Mean Absolute Percent Error)
$$ MAPE = \frac{100\%}{n} \sum_{i=1}^{n} \frac{|y_i - \hat{y}_i|}{|y_i|} $$
#### Преимущества
* Легко интерпретировать.
* Используется в задачах, где неизвестно, какое значение метрики считать приемлемым.
#### Недостатки
* Плохо подходит для задач, в которых важны конкретные единицы измерений.
* Лучше использовать в паре с MAE, чтобы знать абсолютную ошибку и её значение в процентах.

In [20]:
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y, y_predict) * 100))

MAPE score: 113.609 %


### Средняя квадратическая ошибка — MSE (Mean Square Error)
$$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y_i})^2 $$
#### Преимущества
* Каждая ошибка вносит свой квадратичный штраф, большие расхождения между предсказанием и истиной увеличивают штраф.
#### Недостатки
* Измеряется в квадратах единиц, поэтому менее доступна для понимания.
* Искажается при наличии выбросов.
* Не поможет, если нужно сравнить модели, предсказывающие одно и то же по разным признакам.

In [22]:
#Рассчитываем MSE
print('MSE score: {:.3f}'.format(metrics.mean_squared_error(y, y_predict)))

MSE score: 128943244.636


### Корень из средней квадратической ошибки — RMSE (Root Mean Squared Error)
$$ RMSE = \sqrt{MSE} $$
#### Преимущества
* Имеет те же преимущества, что и MSE, но более удобна для понимания (измеряется в тех же единицах, что и целевая переменная).
#### Недостатки
* Не поможет, если нужно сравнить модели, предсказывающие одно и то же по разным признакам.

In [23]:
import numpy as np

#Рассчитываем RMSE
print('RMSE score: {:.3f}'.format(np.sqrt(metrics.mean_squared_error(y, y_predict))))

RMSE score: 11355.318


### Коэффициент детерминации ($R^2$)
$$ R^2 = 1 - \frac{MSE}{MSE_{mean}}, $$
$$ MSE_{mean} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \bar{y_i}), $$
где $\bar{y_i}$ — среднее по вектору правильных ответов.
#### Преимущества
* Можно сравнивать модели, обученные на разных признаках.
* Легко оценить качество модели: измеряется от $-\infty$ до 1.
* Удовлетворительным показателем считается показатель выше 0.5.
#### Недостатки
* Чувствительна к добавлению новых данных.
* Чувствительна к выбросам, так как основана на MSE.

In [24]:
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y, y_predict)))

R2 score: 0.120
